<a href="https://colab.research.google.com/github/MoRushaid/BookRecommendation/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-08-18 10:33:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.08s   

2023-08-18 10:33:31 (323 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df = pd.merge(df_books, df_ratings, on='isbn', how='outer')

In [4]:
df.describe()

,user,rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [5]:
df_books[df_books['author'].isnull()]

,isbn,title,author
187700,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN


In [6]:
user_ratings_count = df_ratings['user'].value_counts()
book_ratings_count = df_ratings['isbn'].value_counts()
#print(user_ratings_count)
#print(book_ratings_count)

In [7]:
valid_users = user_ratings_count[user_ratings_count >= 200].index
valid_books = book_ratings_count[book_ratings_count >= 100].index
#print(valid_users)
#print(valid_books)

In [8]:
df = df[df['user'].isin(valid_users) & df['isbn'].isin(valid_books)]
df.describe()

,user,rating
count,49781.000000,49781.000000
mean,140993.388562,2.024086
std,80901.415891,3.576591
min,254.000000,0.000000
25%,70415.000000,0.000000
50%,140358.000000,0.000000
75%,213350.000000,3.000000
max,278418.000000,10.000000


In [9]:
df.head()

,isbn,title,author,user,rating
98,0440234743,The Testament,John Grisham,277478.0,0.0
103,0440234743,The Testament,John Grisham,2977.0,0.0
105,0440234743,The Testament,John Grisham,3363.0,0.0
108,0440234743,The Testament,John Grisham,7346.0,9.0
112,0440234743,The Testament,John Grisham,9856.0,0.0


In [10]:
pivot_table = df.pivot_table(index='title', columns = 'user', values='rating')
pivot_table = pivot_table.fillna(0)

In [11]:
print(pivot_table)

user                                                254.0     2276.0    \
title                                                                    
1984                                                     9.0       0.0   
1st to Die: A Novel                                      0.0       0.0   
2nd Chance                                               0.0      10.0   
4 Blondes                                                0.0       0.0   
A Beautiful Mind: The Life of Mathematical Geni...       0.0       0.0   
...                                                      ...       ...   
Without Remorse                                          0.0       0.0   
Year of Wonders                                          0.0       0.0   
You Belong To Me                                         0.0       0.0   
Zen and the Art of Motorcycle Maintenance: An I...       0.0       0.0   
\O\" Is for Outlaw"                                      0.0       0.0   

user                                 

In [12]:
model = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute', n_jobs=-1)
model.fit(pivot_table.values)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=6)

In [13]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_index = pivot_table.index.get_loc(book)
  distances, indices = model.kneighbors(pivot_table.iloc[book_index].values.reshape(1, -1), n_neighbors=6)

  recommended_books = []
  for i in range(len(distances.flatten()) - 1, 0, -1):
    recommended_books.append([pivot_table.index[indices.flatten()[i]], distances.flatten()[i]])

  return [book, recommended_books]

In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

  for i in range(2):

    if recommends[1][i][0] not in recommended_books:
      test_pass = False

    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False

  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
